## Deleak a given set of systems
> In this example we will be deleaking posebusters

In [1]:
from pathlib import Path
import pandas as pd
from dataclasses import dataclass, field
from plinder.data.splits import get_default_config
from plinder.data.common.constants import BASE_DIR
from cloudpathlib import GSPath

EXAMPLE_FOLDER = BASE_DIR.parent.parent.parent / "examples"
# data_dir = Path("/Users/yusuf/plinder_local_data/2024-06/v2")
DATA_DIR = GSPath("gs://plinder/2024-06/v2")

@dataclass
class TestCriteria:
    max_entry_resolution: float = 3.5
    max_entry_r: float = 0.4
    max_entry_rfree: float = 0.45
    max_entry_r_minus_rfree: float = 0.05
    ligand_max_num_unresolved_heavy_atoms: int = 0
    ligand_max_alt_count: int = 1
    ligand_min_average_occupancy: float = 0.8
    ligand_min_average_rscc: float = 0.8
    ligand_max_average_rsr: float = 0.3
    ligand_max_percent_outliers_clashes: float = 0
    pocket_max_num_unresolved_heavy_atoms: int = 0
    pocket_max_alt_count: int = 1
    pocket_min_average_occupancy: float = 0.8
    pocket_min_average_rscc: float = 0.8
    pocket_max_average_rsr: float = 0.3
    pocket_max_percent_outliers_clashes: int = 100


@dataclass
class GraphConfig:
    metric: str
    threshold: int  # edges above this threshold are kept
    depth: int  # neighbors at this depth are counted as leakage

@dataclass
class SplitConfig:
    proto_test_criteria: TestCriteria = field(default_factory=lambda: TestCriteria())
    graph_configs: list[GraphConfig] = field(
        default_factory=lambda: [
            GraphConfig("pli_qcov", 30, 1),
            GraphConfig("pocket_qcov", 50, 1),
        ]
    )
    test_cluster_cluster: str = "communities"
    # metric to use for sampling representatives from each component
    test_cluster_metric: str = "pli_qcov"
    # threshold to use for sampling representatives from each component
    test_cluster_threshold: int = 50
    # directed to use for sampling representatives from each component
    test_cluster_directed: bool = False
    cluster_column: str = "cluster"
    # max number of representatives from each community
    num_test_representatives: int = 3
    # test should not be singletons
    min_test_cluster_size: int = 5
    # test/val should not be in too big communities or cause too many train cases to be removed
    max_test_leakage_count: int = 300
    mms_unique_quality_count: int = 3
    test_fraction: float = 0.01

    val_cluster_cluster: str = "components"
    val_cluster_metric: str = "pocket_qcov"  # metric to use for splitting train and val
    val_cluster_threshold: int = 50  # threshold to use for splitting train and val
    val_cluster_directed: bool = False  # directed to use for splitting train and val
    val_fraction: float = 0.01
    min_val_cluster_size: int = 5  # val should not be singletons
    num_val_representatives: (
        int
    ) = 3  # max number of val representatives from each community

cfg =  get_default_config()

posebusters_pdbid_ccd_code = pd.read_csv(EXAMPLE_FOLDER / "data/posebusters_benchmark_set_ids.txt", header=None)[0].to_list()



In [2]:
from plinder.data.splits import deleak_entry_nk_single, prep_data_for_desired_properties, load_graphs, assign_split_membership, get_sampling_clusters
from typing import Any, Dict, List, Set, Tuple
from omegaconf import DictConfig
from tqdm import tqdm
from plinder.core.utils.log import setup_logger
LOG = setup_logger(__name__)

def sort_ccd_codes(ccd_codes_str: str) -> str:
    return "-".join(sorted(ccd_codes_str.split("-")))


def ccd_codes_to_set(ccd_codes_str: str) -> Set[str]:
    return set(ccd_codes_str.split("-"))


def map_pdb_ccd_code_to_system_ids(
    data_dir: Path, pdbid_ccd_codes: Set[str]
) -> Tuple[Dict[str, str], Set[str]]:
    annotations = pd.read_parquet(data_dir / "index/annotation_table.parquet")
    pdbid_ccd_codes = {i.upper() for i in pdbid_ccd_codes}

    # VO: not sure if we need this here
    # annotations["ligand_ccd_code"] = annotations["ligand_ccd_code"].apply(
    #     lambda x: sort_ccd_codes(x)
    # )

    # deal with multiligand cases!
    annotations["ligand_ccd_code"] = annotations["ligand_ccd_code"].apply(
        lambda x: ccd_codes_to_set(x)
    )
    annotations = annotations.explode("ligand_ccd_code")

    annotations["pdbid_ligand_ccd_code"] = (
        annotations["entry_pdb_id"] + "_" + annotations["ligand_ccd_code"]
    )
    annotations["pdbid_ligand_ccd_code"] = annotations[
        "pdbid_ligand_ccd_code"
    ].str.upper()
    annotations = annotations[
        annotations["pdbid_ligand_ccd_code"].isin(pdbid_ccd_codes)
    ]
    pdb_id_ccd_to_system_ids = annotations[
        ["pdbid_ligand_ccd_code", "system_id"]
    ].copy()
    pdb_id_ccd_to_system_ids_dict = (
        pdb_id_ccd_to_system_ids.groupby("pdbid_ligand_ccd_code")
        .first()
        .to_dict()["system_id"]
    )
    not_in_plinder = {
        code for code in pdbid_ccd_codes if code not in pdb_id_ccd_to_system_ids_dict
    }
    LOG.warn(
        f"Could not map {len(not_in_plinder)} out of {len(pdbid_ccd_codes)} systems"
    )
    return pdb_id_ccd_to_system_ids_dict, not_in_plinder


def deleak_specific_test_systems(
    data_dir: Path,
    cfg: DictConfig,
    system_ids_to_deleak: Set[str],
    output_tag: str,
    id_is_pdb_ccd_codes: bool = True,
) -> pd.DataFrame:
    if id_is_pdb_ccd_codes:
        system_ids_to_deleak_dict, not_in_plinder = map_pdb_ccd_code_to_system_ids(
            data_dir, system_ids_to_deleak
        )
        system_ids_to_deleak = set(system_ids_to_deleak_dict.values())

    system_id_to_leakage = {}
    (mms_df, entries, quality, nonredundant_to_all) = prep_data_for_desired_properties(
        data_dir, cfg
    )

    graphs, vertex_mappings, vertex_ids = load_graphs(
        data_dir, set(entries["system_id"]), cfg
    )

    (
        cluster_to_systems,
        system_to_cluster,
        entries,
        test_systems,
        system_to_cluster_val,
    ) = get_sampling_clusters(cfg, data_dir, entries)

    for system_id in tqdm(system_ids_to_deleak):
        if system_id in system_to_cluster:
            system_id_to_leakage[system_id] = deleak_entry_nk_single(
                system_id,
                cluster_to_systems[system_to_cluster[system_id]],
                graphs,
                cfg.split.graph_configs,
                vertex_mappings,
                vertex_ids,
            )

    (data_dir / "splits").mkdir(exist_ok=True)
    split_path = data_dir / "splits" / f"splits_{output_tag}.parquet"
    return assign_split_membership(
        cfg,
        entries,
        nonredundant_to_all,
        system_ids_to_deleak,
        system_id_to_leakage,
        system_to_cluster_val,
        split_path,
    )


In [ ]:
deleaked_posebusters = deleak_specific_test_systems(
        DATA_DIR,
        cfg,
        posebusters_pdbid_ccd_code,
        "posebusters_test",
        id_is_pdb_ccd_codes=True)

In [19]:
deleaked_posebusters

,system_id,uniqueness,split,cluster,cluster_for_val_split
0,3grt__1__1.A__1.B,3grt__A__B_c239137,train,c45,c0
388467,3t60__1__1.C__1.H,3t60__C__H_c13371,train,c154,c82
388466,3t60__1__1.B_1.C__1.F,3t60__B_C__F_c365465,train,c154,c82
388465,3t60__1__1.A__1.D,3t60__A__D_c13371,train,c154,c82
388464,1t6z__1__1.B__1.D,1t6z__B__D_c118538,train,c1412,c1100
...,...,...,...,...,...
490041,5nff__9__1.I__1.KA,5nff__I__KA_c82803,train,c447,c0
590871,8gk3__9__1.I__1.OA_1.PA_1.QA,8gk3__I__OA_PA_QA_c109707,removed,c9,c0
314020,4kxf__9__3.H__3.R,4kxf__H__R_c113350,train,c663,c488
314018,4kxf__9__2.A__2.I,4kxf__A__I_c113351,train,c663,c488
